In [1]:
%load_ext nb_black

<IPython.core.display.Javascript object>

In [2]:
import cv2


def read_image(image_file_path):
    """Reads in the image using openCV"""
    return cv2.imread(image_file_path)

<IPython.core.display.Javascript object>

In [3]:
def lets_see_it(image):
    """Displays the image."""
    cv2.imshow("window name", image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

<IPython.core.display.Javascript object>

In [4]:
def grayscale(image):
    """Grayscales image using openCV."""
    return cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

<IPython.core.display.Javascript object>

In [5]:
import imutils


def normalize_dimensions(image, desired_width=20, desired_height=20):
    """Resizes image to desired dimensions using white padding and rescaling."""
    (h, w) = image.shape[:2]
    if w > h:
        image = imutils.resize(image, width=desired_width)
    else:
        image = imutils.resize(image, height=desired_height)
    width_padding = int((desired_width - image.shape[1]) / 2)
    height_padding = int((desired_height - image.shape[0]) / 2)
    WHITE = [255, 255, 255]
    image_with_border = cv2.copyMakeBorder(
        image,
        height_padding,
        height_padding,
        width_padding,
        width_padding,
        cv2.BORDER_CONSTANT,
        value=WHITE,
    )
    image_with_border_resized = cv2.resize(
        image_with_border, (desired_width, desired_height), interpolation=cv2.INTER_AREA
    )
    return image_with_border_resized

<IPython.core.display.Javascript object>

In [6]:
import numpy as np


def reshape_for_keras(image):
    """Adds a dummy dimension to fit keras's input requirements."""
    return np.expand_dims(image, axis=2)

<IPython.core.display.Javascript object>

In [7]:
from imutils import paths
import os

images = []
labels = []
captcha_processing_output_folder = "extracted_character_images"
for image_file_path in paths.list_images(captcha_processing_output_folder):
    image = read_image(image_file_path)
    image_gray = grayscale(image)
    image_normalized = normalize_dimensions(image_gray)
    image_reshaped_for_keras = reshape_for_keras(image_normalized)
    images.append(image_reshaped_for_keras)
    label = image_file_path.split(os.path.sep)[-2]
    labels.append(label)

<IPython.core.display.Javascript object>

In [8]:
X = np.array(images, dtype="float") / 255.0
labels = np.array(labels)

<IPython.core.display.Javascript object>

In [10]:
from sklearn.preprocessing import LabelBinarizer

label_binarizer = LabelBinarizer().fit(labels)
y = label_binarizer.transform(labels)

<IPython.core.display.Javascript object>

In [11]:
y

array([[1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 1],
       [0, 0, 0, ..., 0, 0, 1],
       [0, 0, 0, ..., 0, 0, 1]])

<IPython.core.display.Javascript object>

In [12]:
from tensorflow.keras import backend
from keras.models import Sequential
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.layers.core import Flatten, Dense

num_classes = len(set(labels))

Using TensorFlow backend.


<IPython.core.display.Javascript object>

In [14]:
CNN_model = Sequential()
CNN_model.add(
    Conv2D(20, (5, 5), padding="same", input_shape=(20, 20, 1), activation="relu")
)
CNN_model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
CNN_model.add(Conv2D(50, (5, 5), padding="same", activation="relu"))
CNN_model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
CNN_model.add(Flatten())
CNN_model.add(Dense(512, activation="relu"))
CNN_model.add(Dense(num_classes, activation="softmax"))

<IPython.core.display.Javascript object>

In [15]:
CNN_model.compile(
    loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"]
)
CNN_model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 20, 20, 20)        520       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 10, 10, 20)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 10, 10, 50)        25050     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 5, 5, 50)          0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 1250)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 512)               640512    
_________________________________________________________________
dense_2 (Dense)              (None, 32)               

<IPython.core.display.Javascript object>

In [16]:
from ann_visualizer.visualize import ann_viz

ann_viz(CNN_model, title="CNN Model Visualization")

<IPython.core.display.Javascript object>

In [17]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=True)

<IPython.core.display.Javascript object>

In [18]:
CNN_model.fit(
    X_train,
    y_train,
    validation_data=(X_test, y_test),
    batch_size=32,
    epochs=5,
    verbose=1,
)

Train on 14908 samples, validate on 3728 samples
Epoch 1/5
14908/14908 [==============================] - 11s 766us/step - loss: 0.5141 - accuracy: 0.8692 - val_loss: 0.0400 - val_accuracy: 0.9925
Epoch 2/5
14908/14908 [==============================] - 11s 729us/step - loss: 0.0427 - accuracy: 0.9905 - val_loss: 0.0279 - val_accuracy: 0.9952
Epoch 3/5
14908/14908 [==============================] - 11s 728us/step - loss: 0.0165 - accuracy: 0.9958 - val_loss: 0.0165 - val_accuracy: 0.9960
Epoch 4/5
14908/14908 [==============================] - 12s 772us/step - loss: 0.0097 - accuracy: 0.9972 - val_loss: 0.0278 - val_accuracy: 0.9925
Epoch 5/5
14908/14908 [==============================] - 11s 729us/step - loss: 0.0056 - accuracy: 0.9985 - val_loss: 0.0105 - val_accuracy: 0.9957


<IPython.core.display.Javascript object>

In [19]:
import pickle

pickle.dump(label_binarizer, open("binarizer.pkl", "wb"))
CNN_model.save_weights("weights.h5")

<IPython.core.display.Javascript object>